# Battle of the Neighborhoods - Week 4
by **Morteza Ghorbani Kari**

### Part 1: Scrape the web & Locate the districts

In [23]:
# Importing libraries
import time
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import folium
from shapely.geometry import Point

In [2]:
def latLng(name):
    """
    input: address
    output: (latitude, longitude)
    """
    address = name
    geolocator = Nominatim(user_agent="Istanbul")
    
    # Trying to get past repeated GeocoderTimedOut and GeocoderUnavailable errors
    try:   
        location = geolocator.geocode(address)
        
    except (GeocoderTimedOut, GeocoderUnavailable):
        time.sleep(1) # Giving it some time
        location = latLng(name)
        
    print("#", end="")
    
    return location

In [3]:
def distance(point1, point2):
    """
    input: two shapely Point objects
    output: distance between the points (in kilometers)
    """
    
    # approximate radius of earth in km
    R = 6373.0
    
    lat1 = radians(point1.x)
    lon1 = radians(point1.y)
    lat2 = radians(point2.x)
    lon2 = radians(point2.y)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [4]:
# Reading the whole webpage content
url = 'https://en.wikipedia.org/wiki/List_of_districts_of_Istanbul'
page = requests.get(url)
raw_data = BeautifulSoup(page.text)

In [5]:
# Extracting the needed data from the contents

raw_table = raw_data.select('td:nth-child(1)')
istanbul_districts = pd.DataFrame(columns=["District", "Latitude", "Longitude"], index=range(39))

ind = 0
for line in raw_table:
    if line.string == " ":
        continue
        
    elif line.a in line:
        istanbul_districts.loc[ind,"District"] = line.a.string.strip()
        ind += 1

istanbul_districts = istanbul_districts.astype({"Latitude": 'float', "Longitude": 'float'})

In [6]:
for index, row in istanbul_districts.iterrows():
    location = latLng(row[0]+', Istanbul')
    istanbul_districts.loc[index, "Latitude"] = location.latitude
    istanbul_districts.loc[index, "Longitude"] = location.longitude
print("\n", end="")
print("Geolocating Districts is done!")

istanbul_districts.head(10)

#######################################
Geolocating Districts is done!


,District,Latitude,Longitude
0,Adalar,40.876259,29.091027
1,Arnavutköy,41.184182,28.740729
2,Ataşehir,40.984749,29.106720
3,Avcılar,40.980135,28.717547
4,Bağcılar,41.033899,28.857898
5,Bahçelievler,41.000290,28.863745
6,Bakırköy,40.983541,28.867974
7,Başakşehir,41.082556,28.800498
8,Bayrampaşa,41.051452,28.898305
9,Beşiktaş,41.042847,29.007528


In [7]:
istanbul_districts.to_csv("Istanbul_Districts.csv", index=False)

In [8]:
data_points = dict()

for index, row in istanbul_districts.iterrows():
    name = row[0]
    point = Point(row[1], row[2]) 
    data_points[name] = point

In [9]:
distances = pd.DataFrame(columns=data_points.keys(), index=data_points.keys(), dtype=float)

for dist1 in data_points.keys():
    for dist2 in data_points.keys():
        distances.loc[dist1, dist2] = distance(data_points[dist1], data_points[dist2])
        
# Replacing 0s so they don't mess up the minimum distance calculation
distances[distances == 0] = None

print(f'The minimum distance between districts is {distances.min().min():.3f} KMs.\nWe will use this distance for our venue search.')

The minimum distance between districts is 1.513 KMs.
We will use this distance for our venue search.


In [10]:
ist_lat = 41.0082
ist_lng = 28.9784

map_istanbul = folium.Map(location=[ist_lat, ist_lng], zoom_start=9)
folium.LayerControl().add_to(map_istanbul)

for lat, lng, district in zip(istanbul_districts['Latitude'], istanbul_districts['Longitude'],
                                                 istanbul_districts['District']):
    label = f'{district}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
                        [lat, lng],
                        radius=5,
                        popup=label,
                        color='#ff0000',
                        fill=True,
                        fill_color='#ffff00',
                        fill_opacity=0.7,
                        parse_html=False,
                        ).add_to(map_istanbul)

map_istanbul

### Part 2: Explore venues

In [11]:
CLIENT_ID = 'AECEZ05OKNUM4DDQZPMRZ3FNDFXEWNFMHKAGCOUAYSCR4CJ1' # your Foursquare ID
CLIENT_SECRET = 'M3EOELPIXED54OA0QFAX0MTJNH3KLW5V1ZBYVOUE0OW4DEYD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100

Your credentails:
CLIENT_ID: AECEZ05OKNUM4DDQZPMRZ3FNDFXEWNFMHKAGCOUAYSCR4CJ1
CLIENT_SECRET:M3EOELPIXED54OA0QFAX0MTJNH3KLW5V1ZBYVOUE0OW4DEYD


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
istanbul_venues = getNearbyVenues(names=istanbul_districts['District'],
                                   latitudes=istanbul_districts['Latitude'],
                                   longitudes=istanbul_districts['Longitude']
                                  )

Adalar
Arnavutköy
Ataşehir
Avcılar
Bağcılar
Bahçelievler
Bakırköy
Başakşehir
Bayrampaşa
Beşiktaş
Beykoz
Beylikdüzü
Beyoğlu
Büyükçekmece
Çatalca
Çekmeköy
Esenler
Esenyurt
Eyüp
Fatih
Gaziosmanpaşa
Güngören
Kadıköy
Kağıthane
Kartal
Küçükçekmece
Maltepe
Pendik
Sancaktepe
Sarıyer
Silivri
Sultanbeyli
Sultangazi
Şile
Şişli
Tuzla
Ümraniye
Üsküdar
Zeytinburnu


In [14]:
istanbul_venues.to_csv('istanbul_venues.csv', index=False)

In [17]:
istanbul_venues.head(10)

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adalar,40.876259,29.091027,İnönü Evi Müzesi,40.878251,29.093647,History Museum
1,Adalar,40.876259,29.091027,Merit Halki Palace Hotel,40.878802,29.090974,Hotel
2,Adalar,40.876259,29.091027,L'isola Guesthouse,40.877038,29.096136,Bed & Breakfast
3,Adalar,40.876259,29.091027,Luz Café,40.877528,29.097877,Café
4,Adalar,40.876259,29.091027,Heybeliada Su Sporları Kulübü,40.882365,29.089167,Pool
5,Adalar,40.876259,29.091027,Türk Kızılayı Heybeliada Gençlik Kampı,40.872446,29.086737,Campground
6,Adalar,40.876259,29.091027,Heybeliada Su Sporları Klübü Havuzu,40.882023,29.090130,Pool
7,Adalar,40.876259,29.091027,Heybeli Sahaf,40.877544,29.097793,Bookstore
8,Adalar,40.876259,29.091027,Sandal,40.871528,29.086537,Beach
9,Adalar,40.876259,29.091027,Aqua Green Beach,40.880498,29.090354,Beach


In [30]:
from folium.plugins import FastMarkerCluster

ist_lat = 41.0082
ist_lng = 28.9784

map_district = folium.Map(location=[ist_lat, ist_lng], zoom_start=9)
map_district.add_child(FastMarkerCluster(istanbul_venues[['Venue Latitude', 'Venue Longitude', 'District Latitude']].values.tolist()))

In [31]:
print('There are {} uniques categories.'.format(len(istanbul_venues['Venue Category'].unique())))

There are 296 uniques categories.


### Part 3: Preprocess the data

In [19]:
istanbul_onehot = pd.get_dummies(istanbul_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
istanbul_onehot['District'] = istanbul_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [istanbul_onehot.columns[-1]] + list(istanbul_onehot.columns[:-1])
istanbul_onehot = istanbul_onehot[fixed_columns]

istanbul_onehot.head()

,District,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Borek Place,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Center,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Caribbean Restaurant,Casino,Castle,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Cigkofte Place,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Classroom,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Conference,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Fishing Spot,Flea Market,Flower Shop,Food,Food Court,Food Truck,Football Stadium,Forest,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Housing Development,Hunting Supply,IT Services,Ice Cream Shop,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Internet Cafe,Irish Pub,Island,Italian Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kofte Place,Kokoreç Restaurant,Kuruyemişçi,Leather Goods Store,Lighthouse,Lingerie Store,Lounge,Manti Place,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Meyhane,Middle Eastern Restaurant,Mobile Phone Shop,Mosque,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Event Space,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pide Place,Pie Shop,Pilates Studio,Pilavcı,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Club,Roller Rink,Roof Deck,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Söğüş Place,Taco Place,Tailor Shop,Tantuni Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tiki Bar,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Turkish Home Cooking Restaurant,Turkish Restaurant,Turşucu,University,Video Store,Volleyball Court,Warehouse Store,Watch Shop,Water Park,Waterfront,Wedding Hall,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zo

#### All Venues (oneHotEncoded by the category) shape

In [25]:
istanbul_onehot.shape

(3134, 297)

In [29]:
istanbul_grouped = istanbul_onehot.groupby('District').mean().reset_index()
istanbul_grouped

,District,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Borek Place,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Center,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Caribbean Restaurant,Casino,Castle,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Cigkofte Place,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Classroom,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Conference,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Fishing Spot,Flea Market,Flower Shop,Food,Food Court,Food Truck,Football Stadium,Forest,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Housing Development,Hunting Supply,IT Services,Ice Cream Shop,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Internet Cafe,Irish Pub,Island,Italian Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kofte Place,Kokoreç Restaurant,Kuruyemişçi,Leather Goods Store,Lighthouse,Lingerie Store,Lounge,Manti Place,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Meyhane,Middle Eastern Restaurant,Mobile Phone Shop,Mosque,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Event Space,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pide Place,Pie Shop,Pilates Studio,Pilavcı,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Club,Roller Rink,Roof Deck,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Söğüş Place,Taco Place,Tailor Shop,Tantuni Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tiki Bar,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Turkish Home Cooking Restaurant,Turkish Restaurant,Turşucu,University,Video Store,Volleyball Court,Warehouse Store,Watch Shop,Water Park,Waterfront,Wedding Hall,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zo

#### All Districts (oneHotEncoded by the category) shape

In [27]:
istanbul_grouped.shape

(39, 297)